In [5]:
import pandas as pd
import os
import numpy as np
import sklearn
import dataFuncs

#Initialise repositories
_projroot = os.path.abspath('.')
_datadir = os.path.join(_projroot,'data')
_preprocesseddir = os.path.join(_datadir,'preprocesseddata')
_rawdir = os.path.join(_datadir,'rawdata')

In [11]:
#Create datasets
from sklearn.model_selection import train_test_split

WCC_Survey = pd.read_csv(os.path.join(_preprocesseddir,'WCCSurvey_mobile.csv'))
WCC_Survey = WCC_Survey.dropna()
WCC_Survey = WCC_Survey.drop('Unnamed: 0', axis = 1)
WCC_Survey = WCC_Survey.drop('case', axis = 1)